1. This code receives KML points generated in Google Earth software and converts them into a dataframe.
2. It calculates the distance between the points to ensure they are created correctly.
3. It saves the paired points that represent the coastlines.

## Import Modules

In [ ]:
import geopandas as gpd
from shapely import Point, LineString
import fiona
from fiona.drvsupport import supported_drivers
import pandas as pd
import numpy as np
supported_drivers['LIBKML'] = 'rw'

### KML to DF

In [ ]:
# Read the KML file and convert them to dataframe
def kml_to_points(kml_filename):
    gdf = gpd.read_file(kml_filename)
    points = (gdf.set_index('Name').geometry
        .apply(lambda x: list(x.coords))
        .explode(ignore_index=False)
        .apply(pd.Series)
        .reset_index()
        .rename(columns={0: 'longitude', 1: 'latitude'}))
    return gdf, points

In [ ]:
kml_filename='data/kml/mes_island.kml'
_, points_island = kml_to_points(kml_filename)

In [ ]:
kml_filename='data/kml/Near_Coast.kml'
_, points_coast = kml_to_points(kml_filename)

## Compute distance and choose the right points.

In [ ]:
# island
points_island = points_island.drop([6,7,17]).reset_index().drop(columns=['index'])

In [ ]:
# island
points_coast = points_coast.drop([10]).reset_index().drop(columns=['index'])

In [ ]:
points_coast_south = points_coast.loc[:11].reset_index()
points_coast_north = points_coast.loc[12:].reset_index()

In [ ]:
for i in range(len(points_island)-1):
    dist = np.sqrt((points_island.loc[i].longitude - points_island.loc[i+1].longitude)**2 + (points_island.loc[i].latitude - points_island.loc[i+1].latitude)**2)
    print(i,dist)

In [ ]:
for i in range(len(points_coast_south)-1):
    dist = np.sqrt((points_coast_south.loc[i].longitude - points_coast_south.loc[i+1].longitude)**2 + (points_coast_south.loc[i].latitude - points_coast_south.loc[i+1].latitude)**2)
    print(i,dist)

In [ ]:
for i in range(len(points_coast_north)-1):
    dist = np.sqrt((points_coast_north.loc[i].longitude - points_coast_north.loc[i+1].longitude)**2 + (points_coast_north.loc[i].latitude - points_coast_north.loc[i+1].latitude)**2)
    print(i,dist)

In [ ]:
# Save KML files to DF
points_island.to_csv('data/csv/points_island.csv')
points_coast_south.to_csv('data/csv/points_coast_south.csv')
points_coast_north.to_csv('data/csv/points_coast_north.csv')

## Create the pairs

In [ ]:
point_df = pd.DataFrame(columns=['name','lat_A','lon_A','lat_B','lon_B'])

for i in range(len(points_island)-1):
    point_df.loc[len(point_df)] = [f'{np.round(points_island.loc[i].latitude,3)}_{np.round(points_island.loc[i].longitude,3)}',
                       np.round(points_island.loc[i].latitude,3),
                       np.round(points_island.loc[i].longitude,3),
                       np.round(points_island.loc[i+1].latitude,3),
                       np.round(points_island.loc[i+1].longitude,3)]
point_df

In [ ]:
for i in range(len(points_coast_south)-1):
    point_df.loc[len(point_df)] = [f'{np.round(points_coast_south.loc[i].latitude,3)}_{np.round(points_coast_south.loc[i].longitude,3)}',
                       np.round(points_coast_south.loc[i].latitude,3),
                       np.round(points_coast_south.loc[i].longitude,3),
                       np.round(points_coast_south.loc[i+1].latitude,3),
                       np.round(points_coast_south.loc[i+1].longitude,3)]
point_df

In [ ]:
for i in range(len(points_coast_north)-1):
    point_df.loc[len(point_df)] = [f'{np.round(points_coast_north.loc[i].latitude,3)}_{np.round(points_coast_north.loc[i].longitude,3)}',
                       np.round(points_coast_north.loc[i].latitude,3),
                       np.round(points_coast_north.loc[i].longitude,3),
                       np.round(points_coast_north.loc[i+1].latitude,3),
                       np.round(points_coast_north.loc[i+1].longitude,3)]
point_df

## Save the dataframe

In [ ]:
# some points needed to be removes as they where out of the considered region.
point_df = point_df.loc[0:33]

In [ ]:
point_df.to_csv('data/csv/merged_points.csv')